In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Glucocorticoid_Sensitivity/GSE15820'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"ZBTB16, a glucocorticoid response gene in acute lymphoblastic leukemia, interferes with glucocorticoid-induced apoptosis"
!Series_summary	"Glucocorticoids (GCs) cause apoptosis in lymphoid lineage cells and are therefore widely used in the therapy of lymphoid malignancies. The molecular mechanisms of the anti-leukemic GC effects are, however, poorly understood. We have previously defined a list of GC-regulated candidate genes by Affymetrix-based whole genome comparative expression profiling in children with acute lymphoblastic leukemia (ALL) during systemic GC monotherapy and in experimental systems of GC-induced apoptosis. ZBTB16, a Zink finger and BOZ-domain containing transcriptional repressor, was one of the most promising candidates derived from this screen. To investigate its possible role in GC-induced apoptosis and cell cycle arrest, we performed conditional over-expression experiments in CCRF-CEM childhood ALL cells. Transgenic ZBTB16 alo

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if the dataset contains gene expression data
is_gene_available = True

# Determine the availability of 'Glucocorticoid_Sensitivity', 'age', and 'gender' data
trait_row = age_row = gender_row = None  # Not available based on sample characteristics dictionary

# Define conversion functions if necessary
def convert_trait(value):
    return None  # No conversion needed as data is not available

def convert_age(value):
    return None  # No conversion needed as data is not available

def convert_gender(value):
    return None  # No conversion needed as data is not available

# Save cohort information
save_cohort_info('GSE15820', './preprocessed/Glucocorticoid_Sensitivity/cohort_info.json', is_gene_available, trait_row is not None)
